# Regression 종합실습 : Car seat sales
유아용 카시트 매출액을 예측해 봅시다.

* 카시트에 대해서 지역 매장 별 매출액을 예측하고자 합니다.

![](https://cdn.images.express.co.uk/img/dynamic/24/590x/child-car-seat-986556.jpg?r=1532946857754)

## 1.환경준비

### (1) Import

In [1]:
#라이브러리들을 불러오자.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import warnings    # 경고메시지 제외
warnings.filterwarnings(action='ignore')

### (2) Data Loading

In [3]:
data_path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/Carseats.csv'
data = pd.read_csv(data_path)

**변수설명**
* Sales - 각 지역 판매량(단위 : 1000개) <== Target
* CompPrice - 각 지역 경쟁사 가격
* Income - 각 지역 평균 소득수준(단위 : 1000달러)
* Advertising - 각 지역, 회사의 광고 예산(단위 : 1000달러)
* Population - 지역 인구수(단위 : 1000명)
* Price - 자사 지역별 판매가격
* ShelveLoc - 진열상태
* Age - 지역 인구의 평균 연령
* Education - 각 지역 교육수준 레벨
* Urban - 매장 도시 지역 여부
* US - 매장이 미국에 있는지 여부

## 2.데이터 이해

* 둘러보기

In [4]:
data.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


## 3.데이터 준비

### (1) 데이터 정리

- feature engineering을 통해 가격 경쟁력 등의 열 추가 가능
    - 추가할 경우 사용한 열은 삭제(다중공선성 문제)

### (2) 데이터분할1 : x, y 나누기

In [5]:
x = data.drop('Sales', axis=1)
y = data.loc[:, 'Sales']

### (3) NA 조치
- NA값 없음

### (4) 가변수화

- 데이터 확인을 통해 범주형, 숫자형 구분 필요

In [6]:
cols = ['ShelveLoc', 'Urban', 'US'] # Education도 추가해야함
x= pd.get_dummies(x, columns=cols, drop_first=True)

### (5) 데이터분할2 : train : validation 나누기

- random_state: random seed 설정

In [8]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3)

### (6) Scaling
KNN 알고리즘을 적용하기 위해서는 스케일링을 해야 합니다.

In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_train_s = scaler.fit_transform(x_train)
x_val_s = scaler.transform(x_val)

In [29]:
from sklearn.preprocessing import StandardScaler

scaler2 = StandardScaler()
x_train_s2 = scaler.fit_transform(x_train)
x_val_s2 = scaler.transform(x_val)

## 4.모델링 : 선형회귀

* 변수를 조절하며 최소 2개 이상의 모델을 생성하고 예측하고 평가해 봅시다.

In [10]:
from sklearn.linear_model import LinearRegression

* 모델1: `Prics` 변수 사용

In [18]:
features = ['Price']
x_train_1 = x_train[features]
x_val_1 = x_val[features]

In [19]:
model1 = LinearRegression()
model1.fit(x_train_1, y_train)
pred1 = model1.predict(x_val_1)

* 모델2: 모든 변수 사용

In [14]:
model2 = LinearRegression()
model2.fit(x_train, y_train)
pred2 = model2.predict(x_val)

## 5.모델링 : KNN

* 하이퍼파라미터를 조절하며 모델을 최소 3가지 이상 생성하시오.

In [15]:
from sklearn.neighbors import KNeighborsRegressor

* 모델3: k=5, euclidean, MinMax Scale

In [20]:
model3 = KNeighborsRegressor()
model3.fit(x_train_s, y_train)
pred3 = model3.predict(x_val_s)

* 모델4: k=5, manhattan, MinMax Scale

In [21]:
model4 = KNeighborsRegressor(metric='manhattan')
model4.fit(x_train_s, y_train)
pred4 = model4.predict(x_val_s)

* 모델5: k=16, euclidean, MinMax Scale

In [22]:
model5 = KNeighborsRegressor(n_neighbors=16)
model5.fit(x_train_s, y_train)
pred5 = model5.predict(x_val_s)

* 모델6: k=16, manhattan, MinMax Scale

In [26]:
model6 = KNeighborsRegressor(n_neighbors=16, metric='manhattan')
model6.fit(x_train_s, y_train)
pred6 = model6.predict(x_val_s)

* 모델7: k=5, euclidean, Standard Scale

In [30]:
model7 = KNeighborsRegressor()
model7.fit(x_train_s2, y_train)
pred7 = model7.predict(x_val_s2)

* 모델8: k=5, manhattan, MinMax Scale

In [31]:
model8 = KNeighborsRegressor(metric='manhattan')
model8.fit(x_train_s2, y_train)
pred8 = model8.predict(x_val_s2)

## 6.성능비교

In [23]:
from sklearn.metrics import *

In [32]:
models = ['LR_Price', 'LR_All', 'KNN_E5', 'KNN_M5', 'KNN_E16', 'KNN_M16', 'KNN_E5(S)', 'KNN_M5(S)']
preds = [pred1, pred2, pred3, pred4, pred5, pred6, pred7, pred8]
rmse, mae, mape = [], [], []

for pred in preds:
    rmse.append(mean_squared_error(y_val, pred, squared=False))
    mae.append(mean_absolute_error(y_val, pred))
    mape.append(mean_absolute_percentage_error(y_val, pred))

result = pd.DataFrame({'Model': models, 'RMSE': rmse, 'MAE':mae, 'MAPE':mape})

In [33]:
result

,Model,RMSE,MAE,MAPE
0,LR_Price,2.486325,2.050641,0.378112
1,LR_All,1.004462,0.793117,0.120321
2,KNN_E5,2.187732,1.743883,0.308908
3,KNN_M5,2.144566,1.779283,0.311258
4,KNN_E16,2.230042,1.792229,0.319642
5,KNN_M16,2.174776,1.750365,0.308362
6,KNN_E5(S),2.187732,1.743883,0.308908
7,KNN_M5(S),2.144566,1.779283,0.311258
